<a href="https://colab.research.google.com/github/pranavkerkar31/5-Day-AI-Agents-Intensive/blob/main/Assignment2_MultiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv -q

In [29]:
from dotenv import load_dotenv
import os
load_dotenv()
print("API Keys Loaded...")
print("KEY FOUND:", os.getenv("GOOGLE_API_KEY") is not None)

API Keys Loaded...
KEY FOUND: False


In [ ]:
pip install google-adk

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


✅ ADK components imported successfully.


In [ ]:
retry_config = types.HttpRetryOptions(
    attempts = 5,
    exp_base = 7,
    initial_delay = 1,
    http_status_codes = [429, 500, 503, 504],
)

In [ ]:
research_agent = Agent(      #Agent 1
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

print("research_agent created.")

research_agent created.


In [ ]:
summarizer_agent = Agent(   # Agent 2
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("summarizer agent created.")

summarizer agent created.


In [ ]:
root_agent = Agent(   # Agent 3
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("root agent created.")

root agent created.


In [ ]:
runner = InMemoryRunner(agent = root_agent)
print('Runner created')

Runner created


In [ ]:
async for event in runner.run_async(
    new_message=types.Content(
        role="user",
        parts=[
            types.Part(
                text="What are the latest advancements in quantum computing and what do they mean for AI?"
            )
        ]
    )
):
    print(event)


TypeError: Runner.run_async() missing 2 required keyword-only arguments: 'user_id' and 'session_id'

Sequential Agent

In [24]:
#Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)

print("outline_agent created....")

outline_agent created....


In [21]:
# Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction = """Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",
)

print("writer agent created....")

writer agent created....


In [23]:
# Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name = "EditorAgent",
    model = Gemini(
        model ="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction = """Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",
)

print("editor agent created....")

editor agent created....


In [27]:
root_agent = SequentialAgent(
    name = "BlobPipeline",
    sub_agents = [outline_agent, writer_agent, editor_agent],
)
print("Sequential Agent Created")


Sequential Agent Created


In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)

Output we get

 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > Hello! OutlineAgent here, ready to help you craft a compelling blog post outline.

Here's a structure for your blog post on the benefits of multi-agent systems for software developers:

## **Headline:** Supercharge Your Code: Why Multi-Agent Systems Are Your Next Developer Superpower

**Introduction Hook:** Imagine a team of tireless, intelligent assistants working in parallel, tackling complex tasks, debugging with uncanny speed, and optimizing your code in ways you never thought possible. This isn't science fiction anymore; it's the reality of multi-agent systems (MAS) and they're poised to revolutionize how software developers work.

---

### **Main Section 1: Enhanced Problem Solving and Complexity Management**

*   **Divide and Conquer, Smarter:** MAS can break down large, intricate problems into smaller, manageable sub-problems, assigning them to specialized agents. This distributed approach makes tackling highly complex systems significantly more feasible and efficient.
*   **Intelligent Delegation:** Developers can leverage agents to handle routine, repetitive, or computationally intensive tasks, freeing up their cognitive bandwidth for higher-level design, creativity, and strategic thinking.
*   **Adaptive and Robust Solutions:** Agents can communicate, coordinate, and adapt their strategies in real-time, leading to more resilient and flexible software that can better handle unexpected changes or failures.

### **Main Section 2: Accelerated Development Cycles and Increased Productivity**

*   **Automated Testing and Debugging:** Agents can be trained to autonomously identify, isolate, and even suggest fixes for bugs, drastically reducing the time spent on manual testing and debugging.
*   **Code Generation and Optimization:** MAS can assist in generating boilerplate code, refactoring existing code, and identifying areas for performance optimization, leading to faster development timelines.
*   **Continuous Learning and Improvement:** Agents can learn from past interactions and data, continuously improving their performance and providing developers with better insights and assistance over time.

### **Main Section 3: Fostering Collaboration and Innovation**

*   **Simulating Complex Interactions:** MAS can be used to model and simulate real-world scenarios with multiple interacting entities, providing developers with a powerful tool for understanding system behavior and user interactions before deployment.
*   **Exploration of Novel Architectures:** The agent-based paradigm encourages thinking about software in terms of decentralized, autonomous units, potentially leading to the development of entirely new and innovative software architectures.
*   **Democratizing Advanced Capabilities:** MAS can abstract away complex algorithms and techniques, making advanced capabilities like AI and machine learning more accessible and easier to integrate into traditional software development.

---

**Concluding Thought:** Multi-agent systems are more than just a new tool; they represent a paradigm shift in how we approach software development. By embracing their power for problem-solving, productivity, and innovation, developers can unlock unprecedented levels of efficiency and creativity, ultimately building better software, faster.
WriterAgent > ## Supercharge Your Code: Why Multi-Agent Systems Are Your Next Developer Superpower

Imagine a team of tireless, intelligent assistants working in parallel, tackling complex tasks, debugging with uncanny speed, and optimizing your code in ways you never thought possible. This isn't science fiction anymore; it's the reality of multi-agent systems (MAS) and they're poised to revolutionize how software developers work.

At their core, MAS offer a powerful approach to **enhanced problem-solving and complexity management**. Think of it as a smarter "divide and conquer." These systems can break down massive, intricate problems into smaller, manageable sub-problems, assigning them to specialized agents. This distributed approach makes tackling highly complex systems significantly more feasible and efficient. Developers can intelligently delegate routine, repetitive, or computationally intensive tasks to these agents, freeing up their own cognitive bandwidth for higher-level design, creativity, and strategic thinking. Furthermore, agents can communicate and adapt their strategies in real-time, leading to more resilient and flexible software.

The impact on **accelerated development cycles and increased productivity** is profound. MAS can automate testing and debugging, autonomously identifying, isolating, and even suggesting fixes for bugs, drastically reducing manual effort. They can also assist in generating boilerplate code, refactoring existing code, and identifying performance bottlenecks, all contributing to faster development timelines. Crucially, these agents can learn from past interactions, continuously improving their performance and providing developers with ever-better insights.

Beyond efficiency, MAS fosters **collaboration and innovation**. They can simulate complex interactions, offering a powerful tool for understanding system behavior before deployment. The agent-based paradigm encourages exploration of novel architectures, thinking in terms of decentralized, autonomous units. This also democratizes advanced capabilities like AI and machine learning, making them more accessible for integration.

Multi-agent systems are more than just a new tool; they represent a paradigm shift. By embracing their power, developers can unlock unprecedented levels of efficiency and creativity, ultimately building better software, faster.
EditorAgent > ## Supercharge Your Code: Why Multi-Agent Systems Are Your Next Developer Superpower

Imagine a team of tireless, intelligent assistants working in parallel, tackling complex tasks, debugging with uncanny speed, and optimizing your code in ways you never thought possible. This isn't science fiction anymore; it's the reality of multi-agent systems (MAS), and they're poised to revolutionize how software developers work.

At their core, MAS offer a powerful approach to **enhanced problem-solving and complexity management**. Think of it as a smarter "divide and conquer." These systems can break down massive, intricate problems into smaller, manageable sub-problems, assigning them to specialized agents. This distributed approach makes tackling highly complex systems significantly more feasible and efficient. Developers can intelligently delegate routine, repetitive, or computationally intensive tasks to these agents, freeing up their own cognitive bandwidth for higher-level design, creativity, and strategic thinking. Furthermore, agents can communicate and adapt their strategies in real-time, leading to more resilient and flexible software.

The impact on **accelerated development cycles and increased productivity** is profound. MAS can automate testing and debugging, autonomously identifying, isolating, and even suggesting fixes for bugs, drastically reducing manual effort. They can also assist in generating boilerplate code, refactoring existing code, and identifying performance bottlenecks, all contributing to faster development timelines. Crucially, these agents can learn from past interactions, continuously improving their performance and providing developers with ever-better insights.

Beyond efficiency, MAS fosters **collaboration and innovation**. They can simulate complex interactions, offering a powerful tool for understanding system behavior before deployment. The agent-based paradigm encourages the exploration of novel architectures, promoting a mindset of decentralized, autonomous units. This also democratizes advanced capabilities like AI and machine learning, making them more accessible for integration.

Multi-agent systems are more than just a new tool; they represent a paradigm shift. By embracing their power, developers can unlock unprecedented levels of efficiency and creativity, ultimately building better software, faster.